In [ ]:
!pip install git+https://github.com/jaxleyverse/jaxley.git@1.0.0

# Part I: Building single-cell models in Jaxley

In this first section of the tutorial, you will learn the basics of Jaxley and how to build single-cell models.

## Before we get started

Before getting started, we first need to access the data on google cloud:

In [ ]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket("sjcabs_2025_data")

# Get access to the morphology file of an HSE cell.
blob = bucket.blob("jaxley/hse_10016_um.swc")
blob.download_to_filename("hse_10016_um.swc")

Some typical imports for using `Jaxley` are as follows:

In [ ]:
from jax import config
config.update("jax_enable_x64", True)  # Use double precision.
config.update("jax_platform_name", "cpu")  # We use only CPU in this tutorial.

import matplotlib.pyplot as plt
import numpy as np

import jaxley as jx

# Pandas tables should be shown in full-width.
import pandas as pd
pd.set_option("display.max_columns", 50)

## Building a model of an HSE cell

In this part of the tutorial, we build a morphologically detailed model of an HSE cell in the optic lobe of a fruit fly. We can read such a cell from an SWC file (this will take around 45 seconds):

In [ ]:
cell = jx.read_swc("hse_10016_um.swc", ncomp=1)

We can visualize this cell with the `cell.vis()` function:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
cell.vis(ax=ax)
ax.set_aspect("equal")

Any `cell` (or `network`) in `Jaxley` tracks its parameters in two `pandas` Dataframes: `nodes` and `edges`. `nodes` contains all the information that we associate with cellular properties in the model and `edges` tracks all the information relevant to synapses.

For example:

In [ ]:
cell.nodes

The units of each of these values can be found [here](https://jaxley.readthedocs.io/en/latest/faq/question_01.html).

### Views

Since `Cell`s can become very complex, Jaxley utilizes `View`s to make working with them easy and intuitive.

The simplest way to navigate Modules is as follows:

In [ ]:
cell.branch(0)

Views behave very similarly to the `Cell`, i.e. `cell.branch(0)` (the 0th branch of the cell) behaves like the `cell` we instantiated earlier. As such, `cell.branch(0)` also has a `nodes` attribute, which keeps track of its part of the cell:

In [ ]:
cell.branch(0).nodes

You can use the `View` to visualize only parts of the cell, or highlight parts in different colors:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
cell.vis(ax=ax, color="k")
cell.branch(96).vis(ax=ax, color="r", linewidth=5.0)
cell.branch(502).vis(ax=ax, color="b", linewidth=5.0)
cell.branch(602).vis(ax=ax, color="g", linewidth=5.0)
_ = ax.set_aspect("equal")

More information on creating views can be found in [this tutorial](https://jaxley.readthedocs.io/en/latest/tutorials/00_jaxley_api.html) on the `Jaxley` documentation website.

### Channels

Next, we turn to simulating this cell. By default `Jaxley` initializes cells without any ion channels in the membrane and simulates the voltage according to the cable equation
$$
C \frac{\text{d}V}{\text{d}t} = \frac{1}{r_a} \frac{\text{d}^2V}{\text{d}x^2},
$$

where $x$ is a location in space, $r_a$ is the axial resistivity, which defines how strongly voltage is attenuated between nearby locations, and $C$ is the capacitance.

To make the cell more interesting, we want to `insert` ion channels. For this purpose `Jaxley` implements `Channel`s that can be inserted into any branch of the cell using the `insert` method of a `Cell` or a `View`. In this example, we can use passive dynamics, which are implemented with the `Leak` channel:

$$
C \frac{\text{d}V}{\text{d}t} = \overline{g}_{\text{leak}} (E_{\text{leak}} - V) + \frac{1}{r_a} \frac{\text{d}^2V}{\text{d}x^2}
$$


In [ ]:
from jaxley.channels import Leak

cell.insert(Leak())  # This inserts a `Leak` channel into all compartments of the cell.

> This is also were `View`s come in handy, as it allows to easily target the insertion of channels to specific compartments, for example:
```python
from jaxley.channels import Na, K
cell.branch(0).insert(Na)
cell.branch(1).insert(K)
```

You can also implement your own channels. This is explained [here](https://jaxley.readthedocs.io/en/latest/tutorials/05_channel_and_synapse_models.html).

### Changing parameters

Next, we have to adjust parameters of the simulation. In particular, this cell has two important free parameters:

- the strength of the leak channel ($\overline{g}_{\text{Leak}}$) defines how much current "leaks" out of the cell at any location (high $\overline{g}_{\text{Leak}} \rightarrow$ lots of leak)
- the axial resistivity describes how strongly neighboring parts of the cell are coupled to each other (high $r_a \rightarrow$ strong attenuation, i.e. weak coupling)

Here, we will use parameters that [Meier & Borst (Fig 3b)](https://www.cell.com/current-biology/fulltext/S0960-9822(19)30398-7) used for a multicompartment model of a CT1 cell. You can change parameters of your `Cell` with the `.set()` method:

In [ ]:
cell.set("axial_resistivity", 400.0)  # Unit: ohm cm.
cell.set("Leak_gLeak", 1 / 8_000)  # Unit: S / cm2
cell.set("Leak_eLeak", -70.0)  # Unit: mV

> Feel free to inspect `cell.nodes` to check whether the `.set()` was successful.

> Feel free to modify other parameters. Have a look at `cell.nodes` for all things that can be changed. For example, you can modify the `capacitance` (default value is `1.0`).

### Stimulating, recording, and simulating the cell

We will now set up a simulation of the network. We define parameters for our simulation and for the stimulus below:

In [ ]:
# Stimulus.
i_delay = 20.0  # ms
i_amp = 0.1  # nA
i_dur = 100.0  # ms

# Duration and step size.
dt = 1.0  # ms
t_max = 200.0  # ms

# The stimulus current. Feel free to plot it: plt.plot(current)
current = jx.step_current(i_delay, i_dur, i_amp, dt, t_max)

We stimulate the neuron in a branch of its dendrite (the blue branch above). We will record the voltage from the same branch, from a neighboring branch (green above), and from the main branch (red above):

In [ ]:
cell.delete_stimuli()
cell.branch(502).stimulate(current)  # branch(502) -> dendrite

cell.delete_recordings()
cell.branch(502).record()  # branch(502) -> dendrite
cell.branch(602).record()  # branch(602) -> neighboring dendrite
cell.branch(96).record()  # branch(96) -> soma fiber

Finally, we can run the simulation and plot the result:

In [ ]:
voltages = jx.integrate(cell, delta_t=dt)

In [ ]:
time_vector = np.linspace(0, t_max, voltages.shape[1])

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
_ = ax.plot(time_vector, voltages[0], color="b")
_ = ax.plot(time_vector, voltages[1], color="g")
_ = ax.plot(time_vector, voltages[2], color="r")
_ = ax.legend(["Dendrite", "Nearby dendrite", "Main branch"])
_ = ax.set_xlabel("Time (ms)")
_ = ax.set_ylabel("Voltage (mV)")

As you can see, the voltage gets strongly attenuated from the stimulation site (blue) to other parts of the morphology.

### Tasks for participants

1) Decrease the leak conductance (see above: `cell.set("Leak_gLeak", 1 / 8_000)`) and see how it impacts the simulation result.  
2) Decrease the axial resistivity (see above: `cell.set("axial_resistivity", 400.0)`) and see how it impacts the simulation result.  
3) Changing both of the above, can you get a stronger response in the main branch?

# Part II: Building a mini-connectome network simulation

In this section of the tutorial, you will learn how to simulate networks models in `Jaxley`.

In [ ]:
from jax import config
config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import pickle
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit, vmap, value_and_grad

import jaxley as jx
from jaxley.channels import Leak

import pandas as pd
pd.set_option("display.max_columns", 50)

## Building a mini-connectome model

For this example, we will use a small sub-connectome which describes a subset of the optic lobe. The network consists of 925 neurons and 7302 synapses. Before getting started, we again must access the connectomic data on google cloud:

In [ ]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket("sjcabs_2025_data")

# Get access to the connectomic data.
blob = bucket.blob("jaxley/malecns_09_optic_lobe_hex_08_meta.feather")
blob.download_to_filename("malecns_09_optic_lobe_hex_08_meta.feather")
blob = bucket.blob("jaxley/malecns_09_optic_lobe_hex_08_simple_edgelist.feather")
blob.download_to_filename("malecns_09_optic_lobe_hex_08_simple_edgelist.feather")

To get started, we will extract the nodes (neurons) and edges (connections) from the `.feather` files and extract some parts (e.g., photoreceptors) that we will use later.

In [ ]:
# Read the nodes and edges of a connectome.
nodes = pd.read_feather("malecns_09_optic_lobe_hex_08_meta.feather")
edges = pd.read_feather("malecns_09_optic_lobe_hex_08_simple_edgelist.feather")

# Reset the indices of the `edges` dataframe (because `pre` and `post`
# use the indices within the _whole_ connectome, not just our small sub-connectome).
id_to_idx = pd.Series(nodes.index.to_numpy(), index=nodes["malecns_09_id"])
id_to_idx.index = id_to_idx.index.astype(edges["pre"].dtype, copy=False)
edges["pre"] = edges["pre"].map(id_to_idx)
edges["post"] = edges["post"].map(id_to_idx)

# Extract the cell-indices of some parts of the connectome which we will use in this simulation.
excitatory_indices = nodes.query("neurotransmitter_predicted == 'acetylcholine'").index.to_numpy()
inhibitory_indices = nodes.query("neurotransmitter_predicted == 'glutamate'").index.to_numpy()
unknown_indices = nodes.query("neurotransmitter_predicted == 'unclear'").index.to_numpy()
t4_indices = nodes.query("cell_type in ['T4a', 'T4b', 'T4c', 'T4d']").index.to_numpy()
photoreceptors_indices = nodes.query("cell_type in ['R1-R6', 'R8_unclear', 'R7_unclear', 'R8y', 'R7y', 'R7p']").index.to_numpy()

Now, let's build a connectome simulation in `Jaxley`. First, we create a network containing as many cells as there are in our connectome:

In [ ]:
point_neuron = jx.Cell()
num_cells = len(nodes)
net = jx.Network([point_neuron for _ in range(num_cells)])

Secondly, we create groups (see also [this tutorial](https://jaxley.readthedocs.io/en/beta/tutorials/06_groups.html)). This is optional, but it makes it easier to interact with the network later on. For this, we will use the indices we defined above:

In [ ]:
net.cell(excitatory_indices).add_to_group("excitatory")
net.cell(inhibitory_indices).add_to_group("inhibitory")
net.cell(unknown_indices).add_to_group("unkonwn")
net.cell(t4_indices).add_to_group("t4")
net.cell(photoreceptors_indices).add_to_group("photoreceptors")

After having defined the groups, we can now, for example, easily inspect all photoreceptors:

In [ ]:
net.photoreceptors.nodes[:5]

At this point, the network is not yet connected by any synapses. Let's fix this by using the `edges` dataframe and the `connect` method of `Jaxley`:

In [ ]:
from jaxley.synapses import DynamicSynapse
from jaxley.connect import connect

pre_inds = edges["pre"].to_numpy()
post_inds = edges["post"].to_numpy()

# Create views. You can inspect them via `pre_cells.nodes` and `post_cells.nodes`.
pre_cells = net.select(nodes=pre_inds)
post_cells = net.select(nodes=post_inds)

# Connect the network with synapses.
connect(pre_cells, post_cells, DynamicSynapse())

Above, we used a `DynamicSynapse`, which implements the following synapse model:
    
$$
I = \overline{g} \cdot s \cdot (E - V_{\text{post}})
$$
$$
\tau_s \frac{\text{d}s}{\text{d}t} = \sigma\Big((V_{\text{pre}} - V_{\text{thr}}) / \Delta \Big)- s
$$
`Jaxley` offers many other options, see [here](https://jaxley.readthedocs.io/en/beta/jaxley.html#synapses).

You can inspect synapses and their parameters via the `.edges` attribute:

In [ ]:
net.edges

Next, we have to specify single neuron dynamics. For this example, we will again use a passive model and insert only a `Leak` channel into all neurons of our network:

In [ ]:
net.insert(Leak())

## Recordings and stimuli

We have now set up our first connectome model. Before simulating it, we again have to tell `Jaxley` which neurons should be stimulated (and how), which neurons should be recorded from, and what the parameters of the simulation should be (e.g., synaptic strenghts, $\overline{g}_{\text{Leak}}$). In this example, we will record from T4 neurons:

In [ ]:
net.delete_recordings()
net.t4.record()

We will simulate the response to visual stimuli. To mimic these stimuli, we stimulate the photoreceptors in our network:

In [ ]:
# Each stimulus is a step current.
i_delay = 200.0  # ms
i_dur = 500.0  # ms
dt = 10.0  # ms
t_max = 1_000.0  # ms

# The amplitude of each step current.
_ = np.random.seed(0)
i_amps = np.random.rand(len(photoreceptors_indices)) * 0.01
stimuli = jx.datapoint_to_step_currents(i_delay, i_dur, i_amps, dt, t_max)

# Stimulate the network.
net.delete_stimuli()
net.photoreceptors.stimulate(stimuli)

Let's also change some of the parameters. We first modify synaptic parameters:

In [ ]:
net.set("DynamicSynapse_s", 0.0)
net.set("DynamicSynapse_gS", 1e-5)
net.set("DynamicSynapse_v_th", -32.0)

net.excitatory.set("DynamicSynapse_e_syn", 10.0)
net.inhibitory.set("DynamicSynapse_e_syn", -70.0)
net.unkonwn.set("DynamicSynapse_e_syn", -30.0)

> For more details on how to flexibly set synaptic parameters (e.g., by cell type, or by pre-synaptic cell index,...), see [this tutorial](https://jaxley.readthedocs.io/en/latest/tutorials/09_advanced_indexing.html).

We then also modify the passive parameters of our neurons:

In [ ]:
input_resistances = 10_000
net.set("Leak_gLeak", 1 / input_resistances)
net.set("Leak_eLeak", -65.0)
net.set("v", -60.0)

> After modiying parameters, you can also inspect `net.nodes` (for cellular parameters) and `net.edges` (for synaptic parameters) to check if setting them worked as expected.

> You can also modify parameters of individual neurons by doing, for example: `net.cell(0).set("Leak_gLeak", 1e-4)`, or of inidividual synapses by doing `net.DynamicSynapse.edge(0).set("DynamicSynapse_s", 0.01)`.

We can now simulate the voltage of T4 neurons in response to our "visual" stimuli:

In [ ]:
voltages = jx.integrate(net, delta_t=dt)

Let's plot the voltage response of five of the recorded T4 cells:

In [ ]:
time_vector = np.linspace(0, t_max, voltages.shape[1])

fig, ax = plt.subplots(1, 1, figsize=(20, 3))
for neuron_id in range(5):
    _ = ax.plot(time_vector + neuron_id * 1200, voltages[neuron_id], color="k")
_ = ax.set_xlabel("Time (ms)")
_ = ax.set_ylabel("Voltage (mV)")

How can we tell whether this makes sense? Let's say we have obtained activity recordings from the T4 neurons in this network, given the same stimuli. Let's load these activity recordings:

In [ ]:
blob = bucket.blob("jaxley/optic_lobe_v_recordings.pkl")
blob.download_to_filename("optic_lobe_v_recordings.pkl")

with open("optic_lobe_v_recordings.pkl", "rb") as handle:
    recordings = pickle.load(handle)

>  In this example, the activity recordings were generated by another simulation, so they are not "real" experimental recordings.

Let's compare the simulated voltage to these recordings:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 3))
for neuron_id in range(5):
    _ = ax.plot(time_vector + neuron_id * 1000, voltages[neuron_id], color="k")
    _ = ax.plot(time_vector + neuron_id * 1000, recordings[neuron_id], color="b")
_ = ax.set_xlabel("Time (ms)")
_ = ax.set_ylabel("Voltage (mV)")

The match is clearly not perfect yet. In the rest of this tutorial, we will use gradient descent to optimize parameters of this network such that the voltage traces match.

### Tasks for participants

Modify the synaptic conductances (e.g., `DynamicSynapse_gS`) or the cellular parameters (`Leak_gLeak` and `Leak_eLeak`). Inspect `net.nodes` and `net.edges` to see if things worked. Also check how the modifications impacts the modelled voltages. Can you make the fit to the voltage recordings better?

> ⚠️ **_IMPORTANT_**: before moving on to the next parts of this tutorial, reset parameter values as follows:

In [ ]:
net.set("DynamicSynapse_s", 0.0)
net.set("DynamicSynapse_gS", 1e-5)
net.set("DynamicSynapse_v_th", -32.0)

net.excitatory.set("DynamicSynapse_e_syn", 10.0)
net.inhibitory.set("DynamicSynapse_e_syn", -70.0)
net.unkonwn.set("DynamicSynapse_e_syn", -30.0)

input_resistances = 10_000
net.set("Leak_gLeak", 1 / input_resistances)
net.set("Leak_eLeak", -65.0)
net.set("v", -60.0)

# Part III: Training the connectome model

In the final part of this tutorial, you will learn how to train the network model such that it fits "experimental" activity recordings. This includes the following:

- compute the gradient with respect to parameters  
- use parameter transformations  
- define optimizers  

### Defining trainable parameters

To train such a network, we first have to indicate which values should be optimized. For this, we use the `.make_trainable()` method.

To make a single parameter trainable, run `make_trainable` on `net`. For example, for the leak reversal potential:

In [ ]:
net.delete_trainables()

net.make_trainable("Leak_eLeak")

Similarly, we also use a single (shared) parameter for the synaptic reversal potential of each cell type, and for the synaptic threshold:

In [ ]:
net.excitatory.make_trainable("DynamicSynapse_e_syn")
net.inhibitory.make_trainable("DynamicSynapse_e_syn")
net.unkonwn.make_trainable("DynamicSynapse_e_syn")

net.make_trainable("DynamicSynapse_v_th")

We can also define parameters for individual cells or synapses. To do this, use the `"all"` key. The following defines a separate parameter for the leak conductance of every cell in the network:

In [ ]:
net.select(nodes="all").make_trainable("Leak_gLeak")

And the following defines a single parameter for every synapse:

In [ ]:
net.select(edges="all").make_trainable("DynamicSynapse_gS")

> For more flexile ways to share parameters (e.g., by cell-type), check out [this tutorial](https://jaxley.readthedocs.io/en/latest/tutorials/10_advanced_parameter_sharing.html).

Once all parameters are defined, you can use `.get_parameters()` to obtain all trainable parameters.

In [ ]:
params = net.get_parameters()

> Feel free to inspect the `params`. It contains one value for every trainable parameter.

### Defining a loss function

Before we define the loss, we compute the mean and standard deviations of the recorded voltages. These will be used such that the loss function is computed on outputs that are (roughly) standardized.

In [ ]:
rec_mean = jnp.mean(recordings)
rec_std = jnp.std(recordings)
recordings_z_scored = (recordings - rec_mean) / rec_std

In addition, we also have to standardize (and possibly constrain) parameters. To do so, `Jaxley` provides parameter transformations:

In [ ]:
import jaxley.optimize.transforms as jt

transform = jx.ParamTransform([
    {"Leak_eLeak": jt.SigmoidTransform(-70.0, -60.0)},
    {"DynamicSynapse_e_syn": jt.SigmoidTransform(-15.0, 15.0)},
    {"DynamicSynapse_e_syn": jt.SigmoidTransform(-80.0, -60.0)},
    {"DynamicSynapse_e_syn": jt.SigmoidTransform(-70.0, 0.0)},
    {"DynamicSynapse_v_th": jt.SigmoidTransform(-35.0, -25.0)},
    {"Leak_gLeak": jt.SigmoidTransform(5e-5, 2e-4)},
    {"DynamicSynapse_gS": jt.SigmoidTransform(0.0, 0.00002)},
])

These transformations are then used to standardize the parameters:

In [ ]:
opt_params = transform.inverse(params)

We then define the loss: the mean square error between model voltages and experimental recordings:

In [ ]:
def loss(opt_params):
    params = transform.forward(opt_params)
    trace = jx.integrate(net, params=params, delta_t=dt)
    prediction = (trace - rec_mean) / rec_std
    return jnp.mean((prediction - recordings_z_scored) ** 2)

jitted_grad = jit(value_and_grad(loss))

Note that, before passing parameters to the simulation (which is done by `simulate`), we de-standardize the parameters by running `params = transform.forward(opt_params)`.

We can use `JAX`'s inbuilt functions to take the gradient through the entire simulation:

In [ ]:
prediction, gradient = jitted_grad(opt_params)

> Feel free to inspect the `gradient`. It contains one value for every trainable parameter and has the exact same structure as `params`.

### Training

We will use the ADAM optimizer from the [optax library](https://optax.readthedocs.io/en/latest/) to optimize the free parameters:

In [ ]:
import optax

optimizer = optax.adam(learning_rate=0.001)
opt_state = optimizer.init(opt_params)

### Training loop

That's it! We can now turn to training this network:

In [ ]:
for iteration in range(1_000):
    loss_val, gradient = jitted_grad(opt_params)
    updates, opt_state = optimizer.update(gradient, opt_state)
    opt_params = optax.apply_updates(opt_params, updates)

    if iteration % 50 == 0:
        print(f"iteration {iteration}, loss {loss_val}")

final_params = transform.forward(opt_params)

### Evaluation

Once training has finished, we can evaluate the trained network. To this end, we simulate the model with the trained parameters and compare the simulated voltages to the experimental recordings.

In [ ]:
voltages = jx.integrate(net, params=final_params, delta_t=dt)

time_vector = np.linspace(0, t_max, voltages.shape[1])

fig, ax = plt.subplots(1, 1, figsize=(20, 3))
for neuron_id in range(5):
    _ = ax.plot(time_vector + neuron_id * 1000, voltages[neuron_id], color="k")
    _ = ax.plot(time_vector + neuron_id * 1000, recordings[neuron_id], color="b")
_ = ax.set_xlabel("Time (ms)")
_ = ax.set_ylabel("Voltage (mV)")

Indeed, the network now fits the recordings much better! Congrats!

### Tasks for participants

Increase the number of epochs that the network is trained for. Observe how the final accuracy changes when training for more epochs.  

## A note on identifyability

Since our activity recordings were generated by a simulation, we can inspect whether the learned parameters match the _actual_ parameters of the neural system. Let's first load the paramters that were used to generate the activity measurement:

In [ ]:
blob = bucket.blob("jaxley/optic_lobe_input_resistances.pkl")
blob.download_to_filename("optic_lobe_input_resistances.pkl")

blob = bucket.blob("jaxley/optic_lobe_syn_strenghts.pkl")
blob.download_to_filename("optic_lobe_syn_strenghts.pkl")

with open("optic_lobe_input_resistances.pkl", "rb") as handle:
    input_resistances = pickle.load(handle)

with open("optic_lobe_syn_strenghts.pkl", "rb") as handle:
    syn_strenghts = pickle.load(handle)

For easier inspection, we write the trained parameters into the `net.nodes` and `net.edges` dataframes:

In [ ]:
net.write_trainables(final_params)

We can then plot the learned parameters against the true parameters:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
_ = ax[0].scatter(syn_strenghts * 1e6, net.edges["DynamicSynapse_gS"] * 1e6)
_ = ax[1].scatter(input_resistances, 1 / net.nodes["Leak_gLeak"])
_ = ax[0].set_xlabel("True g_syn")
_ = ax[0].set_ylabel("Learned g_syn")

_ = ax[0].set_title(f"rho = {np.corrcoef(syn_strenghts, net.edges["DynamicSynapse_gS"])[0, 1]:.4f}")
_ = ax[1].set_title(f"rho = {np.corrcoef(input_resistances, 1 / net.nodes["Leak_gLeak"])[0, 1]:.4f}")
_ = ax[1].set_xlabel("True input resistance")
_ = ax[1].set_ylabel("Learned input resistance")
plt.subplots_adjust(wspace=0.5)

Unfortunately, there is almost no correlation! This highlights that, while activity recordings might match, the _mechanisms_ used by our model might not yet be correct. There are many possibilities to alleviate this:
- One might train the model on larger datasets.
- One might use measurements of the parameters to constrain them (for example, use synaptic size to constrain the synaptic weight).
- One might share parameters if their values are believed to be similar (thereby reducing the total number of parameters).

# Summary

In this tutorial, we introduced Jaxley and how to build biophysical models. Further  information and features can be found in our tutorials online:

- [Basic tutorials](https://jaxley.readthedocs.io/en/latest/tutorials.html)
- [Advanced tutorials](https://jaxley.readthedocs.io/en/latest/advanced_tutorials.html)

For building connectome models, we believe that the following tutorials should be most helpful:
- [Building custom synapse and channel models](https://jaxley.readthedocs.io/en/latest/tutorials/05_channel_and_synapse_models.html)
- [Defining cell types or other groups](https://jaxley.readthedocs.io/en/latest/tutorials/06_groups.html)
- [Setting synaptic parameters](https://jaxley.readthedocs.io/en/latest/tutorials/09_advanced_indexing.html)
- [Sharing synaptic parameters](https://jaxley.readthedocs.io/en/latest/tutorials/10_advanced_parameter_sharing.html)

If you have any further questions, feel free to post on our [discussion forum](https://github.com/jaxleyverse/jaxley/discussions). Happy coding!